In [1]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy import stats
import seaborn as sns
from datetime import datetime
import datetime
import math
import matplotlib.gridspec as gridspec
from IPython.display import HTML

In [2]:
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['DejaVu Sans']

In [3]:
os.chdir('D:\\Moji fajlovi\\Master\\prog\\Project 5 - Agriculture Data Analysis')

In [4]:
data = pd.read_csv('database_biosense.csv',header=0, delimiter=';',skiprows=1)
data.head()

,Parcel,Area,Year,Crop,Seed production,Variety,Previous crop,Seed units,Manure,Fertiliser amount,...,Irrigation,Harvest,Seed,Fertiliser,Pesticide,Yield,Price,Revenue,Planting date,Harvest date
0,G-1/1,"63,38",2018,Maize,0,NS 205,Maize,184,0,"237,719575",...,9000,"11853,74",9450,"5037,277795","15202,25308","7,811454718","14,4","112484,9479",15.6.2018,15.10.2018
1,G-1/2,"23,01",2018,Barley,0,Salamandre,Maize,46,0,"238,6363636",...,0,"11762,74","8594,30605","7079,225979",1435,"7,146352313","15,8","112912,3665",28.9.2017,5.6.2018
2,G-2/1,"94,88",2018,Soybean,0,Gorstak,Maize,"100,33",0,"214,6331148",...,9000,"11549,5","10191,21689","7146,257137","11843,41404","3,388877141","37,35","126574,5612",19.4.2018,13.9.2018
3,G-2/2,"20,59",2018,Soybean,0,Wendy,Maize,"93,15",0,"140,8450704",...,0,"11571,08","5428,848956","4776,056338","12594,37882","3,71296746","37,35","138679,3346",17.4.2018,14.9.2018
4,G-3/1,"63,62",2018,Maize,0,DKC 5830,Wheat,60,"1,886199308","474,6934926",...,9000,"12801,04","19899,77051","19745,51399","4947,696479","14,60232631","14,4","210273,4989",17.4.2018,14.9.2018


In [5]:
data.isnull().any()

Parcel                    False
Area                      False
Year                      False
Crop                      False
Seed production           False
Variety                    True
Previous crop              True
Seed units                 True
Manure                    False
Fertiliser amount         False
Pesticide amount          False
Tillage                   False
Sowing                    False
Fertiliser application    False
Pesticide application     False
Irrigation                False
Harvest                   False
Seed                      False
Fertiliser                False
Pesticide                 False
Yield                     False
Price                     False
Revenue                   False
Planting date              True
Harvest date               True
dtype: bool

In [6]:
"""I filled 'Seed units' column because of later converting of elements into numbers"""
data.fillna({'Variety':'None','Previous crop': 'None', 'Seed units': '0'}, inplace=True)

In [7]:
convert_todot = lambda x: float(x.replace(',','.'))

In [8]:
data['Area'] = data['Area'].apply(convert_todot)

In [9]:
for i in list(data)[7:23]:
    try:
        data[i] = data[i].apply(convert_todot) #converting all columns expect one, which i will later process
    except ValueError:
        continue

In [10]:
drop = data['Planting date'].dropna()
i1 = list(drop.index.values)
dfdates = data.loc[i1] #I only pick rows in which planting (harvest, below) date exists

In [11]:
drop2 = dfdates['Harvest date'].dropna()
i2 = list(drop2.index.values)
dfdates = dfdates.loc[i2]

In [12]:
convert_date = lambda x: datetime.datetime.strptime(str(x), '%d.%m.%Y')

In [13]:
dfdates['Planting date'] = dfdates['Planting date'].apply(convert_date) #converting elements of column into datetime type
dfdates['Harvest date'] = dfdates['Harvest date'].apply(convert_date)

In [14]:
crops = list(data['Crop'].unique())
crops #list of crops in my database

['Maize',
 'Barley',
 'Soybean',
 'Wheat',
 'Sunflower',
 'Oilseed rape',
 '2nd soybean']

In [15]:
n = len(crops)

In [ ]:
def plot_ph_date(dfdates,i):
    crop = list(dfdates['Crop'].unique())
    fig = plt.figure(figsize=(12,10))
    gs = gridspec.GridSpec(2,2) # i will have three histograms
    df = dfdates[dfdates['Crop']==crop[i]]
    
    dates = ['Planting date','Harvest date']
    
    ax1 = fig.add_subplot(gs[0,:]) #for the biggest histogram
    ax1.hist(df['Planting date'],color='darkgreen')
    ax1.hist(df['Harvest date'],color='darkorange')
    
        
    index_planting = [k for k in df[dates[0]].index.values if df.loc[k,dates[0]].year == 2017] #pick index where year is 2017.
    index_harvest = [k for k in df[dates[1]].index.values if df.loc[k,dates[1]].year == 2017] # same just year for harvest date
    title = '2017.' #title would have '2017.' except if title get change
    
    if crop[i] == 'Barley':
        """Barley is planted in 2017. and harvested in 2018., so i need to find index of Harvest date where year is 2018.
        which I will later plot."""
        index_harvest = [k for k in df[dates[1]].index.values if df.loc[k,dates[1]].year == 2018]
        ax1.clear() #clear the biggest histogram, because it don't show one of dates --> change bins.
        ax1.hist(df['Planting date'],color='darkgreen')
        ax1.hist(df['Harvest date'],bins=2,color='darkorange')
        title = '2018.'
    if crop[i] == '2nd soybean':
        ax1.clear() #similar, clear to change bins 
        ax1.hist(df['Planting date'],bins=4,color='darkgreen')
        ax1.hist(df['Harvest date'],bins=4,color='darkorange')
    ax1.set_ylabel('counts',fontsize=16)
    ax1.legend()
    ax1.grid(axis='y',color='lightgrey',alpha=0.5)
    ax1.set_facecolor('w')
    
    #setting yticks with condition, if interval (min(y),max(y)) is too small then step is 1, otherwise is 4
    y = ax1.get_yticks()
    if max(y) < 8:
        yint = range(math.floor(min(y)), math.ceil(max(y)))
    else:
        yint = range(math.floor(min(y)), math.ceil(max(y))+1,4)
    ax1.set_yticks(yint)
    
    #picking dates for plotting
    p_dates = df[dates[0]].loc[index_planting]
    h_dates = df[dates[1]].loc[index_harvest]
    
    #plot planting date, on the left side of figure
    ax = fig.add_subplot(gs[1,0])
    ax.hist(p_dates.values,color='darkgreen')
    ax.set_title('Distribution of '+dates[0]+' for 2017. year')
    ax.set_ylabel('counts',fontsize=10)
    ax.set_xlabel(dates[0], fontsize=10)
    ax.grid(axis='y',color='lightgrey',alpha=0.5)
    ax.set_facecolor('w')
    
    #setting yticks with condition, if interval (min(y),max(y)) is too small then step is 1, otherwise is 2
    y = ax.get_yticks()
    if max(y) <= 5:
        yint = range(math.floor(min(y)), math.ceil(max(y)))
    else:
        yint = range(math.floor(min(y)), math.ceil(max(y))+1,2)
    ax.set_yticks(yint)
    
    for tick in ax.get_xticklabels(): #rotate dates
        tick.set_rotation(55)
    
    #plot harvest date, on the right side of figure
    ax2 = fig.add_subplot(gs[1,1])
    ax2.hist(h_dates.values,color='darkorange')
    ax2.set_title('Distribution of '+dates[1]+' for '+title+' year') #if it plot barley then title variable is '2018.'
    ax2.set_ylabel('counts',fontsize=10)
    ax2.set_xlabel(dates[1], fontsize=10)
    ax2.grid(axis='y',color='lightgrey',alpha=0.5)
    ax2.set_facecolor('w')
    
    y = ax2.get_yticks()
    if max(y) <= 5:
        yint = range(math.floor(min(y)), math.ceil(max(y)))
    else:
        yint = range(math.floor(min(y)), math.ceil(max(y))+1,2)
    ax2.set_yticks(yint)
    
    for tick in ax2.get_xticklabels():
        tick.set_rotation(55)
    
    #setting xticks with interval (min(x)-10days,max(x)+10days) with step 1day
    pd = np.arange(p_dates.min()-datetime.timedelta(days=10),p_dates.max()+datetime.timedelta(days=10),datetime.timedelta(days=1)) 
    hd = np.arange(h_dates.min()-datetime.timedelta(days=10),h_dates.max()+datetime.timedelta(days=10),datetime.timedelta(days=1))

    ax.set_xticklabels([str(d)[:10] for d in pd])
    ax2.set_xticklabels([str(d)[:10] for d in hd])
    
    fig.align_labels()
    fig.suptitle(crop[i], fontsize = 30, y=0.99, fontweight='bold')
    fig.tight_layout()
    plt.subplots_adjust(top=0.92, bottom=0.08, left=0.10, right=0.95, hspace=0.3, wspace=0.35)
    #plt.savefig(crop[i]+".png", dpi=100,bbox_inches='tight',transparent=True)
    plt.show()
    

In [ ]:
plot_ph_date(dfdates,0)

In [ ]:
#plot for every crop from crops
for i in range(n):
    plot_ph_date(dfdates,i)

In [ ]:
def plot_yield(data,i):
    crop = list(data['Crop'].unique())
    
    df2 = data[data['Crop']==crop[i]]
    
    df16 = df2[df2['Year'] == 2016]
    std16 = np.std(df16['Yield'])
    df17 = df2[df2['Year'] == 2017]
    std17 = np.std(df17['Yield'])
    df18 = df2[df2['Year'] == 2018]
    std18 = np.std(df18['Yield'])    
    
    box_line_col = ['steelblue','darkorange','darkgreen'] #palette for whiskers and other lines around box
    
    #plot years which have more then two lines in database for that crop, otherwise std is 0, and plot is line instead of box
    #also pick enough colors for lines on plot
    if df16.shape[0] <= 2:
        df2 = df2.query('Year == 2017 or Year == 2018')
        box_line_col = ['steelblue','darkorange']
    if df17.shape[0] <= 2:
        df2 = df2.query('Year == 2016 or Year == 2018')
        box_line_col = ['steelblue','darkorange']
        if df16.shape[0] <= 2:
            box_line_col = ['steelblue']
            
    if df18.shape[0] <= 2:
        df2 = df2.query('Year == 2017 or Year == 2016')
        box_line_col = ['steelblue','darkgreen']
        if df17.shape[0] <= 2:
            box_line_col = ['darkorange']
    
    #if for all years we have few data
    if df2.shape[0] <= 2:
        fig = plt.figure(figsize=(10,1))
        ax = fig.add_subplot(111)
        ax.text(0.1,1.5,'A little bit of data for '+crop[i]+'! Please expand your database.',fontsize=20,fontweight='bold',color='r')
        plt.axis('off')
    else:
        fig2 = plt.figure(figsize=(10,5))
        ax = fig2.add_subplot(1,1,1)
        palette = ["lightsteelblue", "navajowhite", "lightgreen"]
        sns.boxplot(x='Year',y='Yield',data=df2,palette=palette)
        ax.grid(axis='y',color='darkgray')
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['left'].set_color('darkgray')
    
    #coloring all lines on plot
        for j,col in enumerate(box_line_col):
            mybox = ax.artists[j]
            mybox.set_edgecolor(col)
            for k in range(j*6,j*6+6):
                line = ax.lines[k]
                line.set_color(col)
                line.set_mfc(col)
                line.set_mec(col)
    
        text = '\n'.join(('Variability of yield:',"\u2022 std 2016: "+str(round(std16,2)), '\u2022 std 2017: '+str(round(std17,2)),
                      '\u2022 std 2018: '+str(round(std18,2))))
        props = dict(boxstyle='round',facecolor='white', edgecolor='darkgreen', alpha=0.5,pad=0.5)
        plt.figtext(0.99, 0.4,text,family='sans-serif', color='k', fontsize=13, bbox=props)
    
        fig2.tight_layout()
        fig2.suptitle(crop[i],x=0.5,y=1.05,fontsize=20,fontweight='bold')
        #plt.savefig(crop[i]+"_yield.png", dpi=100,bbox_inches='tight',transparent=True)
        plt.show()

In [ ]:
plot_yield(data,6)

In [ ]:
#plot boxplots for all crops
for i in range(n):
    plot_yield(data,i)

In [ ]:
#function for percent on pie chart
def func(pct, allvals):
    percent = float((100*pct)/allvals.sum())
    return "{:.2f}%".format(pct,percent)

In [ ]:
for i in range(n):
    
    df5 = data[data['Crop'] == crops[i]]
    length = len(df5['Variety'].unique())
    
    #plot only crops with more than 4 varieties
    if length>4:
        fig5 = plt.figure(figsize=(8,8))
        ax5 = fig5.add_subplot(111)
    
        vc = df5['Variety'].value_counts()
        sort = vc.sort_values(ascending=False)
        ssum = sort.sum()
        pie = sort.iloc[:5]
        pie = pie.append(pd.Series([ssum-pie.sum()],index=['Others']))
        
        wedges, texts, autotexts = ax5.pie(x=pie,autopct=lambda pct: func(pct, pie),radius=0.8)
        plt.setp(autotexts,fontsize=13,fontweight='bold', color='w')
        
        ax5.legend(loc=0,labels=list(pie.index.values),fontsize=15)
    
        props = dict(facecolor='white',edgecolor='dimgrey',alpha=0.5,pad=10)
        text = 'Planted unique varieties: '+str(length)
        plt.figtext(0.1,0.87,text,bbox=props,fontsize=15,va='top', ha='left')
        
        ax5.axis('off')
        
        fig5.suptitle(crops[i],x=0.5,y=1,fontsize=40,fontweight='medium')
        #plt.savefig(crops[i]+"_varieties.png", dpi=360,transparent=True)
        fig5.tight_layout()
plt.show()

In [ ]:
pivot = data[['Crop','Previous crop']].pivot_table(index='Previous crop', columns='Crop',aggfunc=len,fill_value=0,margins=True)
pivot

In [ ]:
#style with css
styles = [dict(selector="th", props=[("font-size", "100%"),("text-align", "center"),("background-color","olivedrab"),("border-left","0.2px solid black")]),
          dict(selector="caption", props=[("text-align","center"),("font-weight","bold"),("font-size","150%")]),
         dict(selector="td", props=[("font-size", "100%"),("text-align", "center"),("background-color","#cce59a"),("border","0.2px solid black")]),
          dict(selector="tbody tr:last-child", props=[("font-weight","bold")]),
          dict(selector="tr td:last-child", props=[("font-weight","bold")]),
          dict(selector="th:first-child", props=[("border","0.2px solid black")]),
         dict(selector='',props=[("border-collapse","collapse"),("border","0.2px solid black")])]

styler = pivot.style.set_table_styles(styles).set_caption('Pervious crop - crop pivot table')

styler

In [ ]:
#test for independence of Previous crop and Crop attribute; p-value,q-degree of freedom
chi2,p,q,expected = stats.chi2_contingency(pivot)
chi2,p,q

In [ ]:
#functions that will be used for plot
def cdf(x,q):
    return 1/(math.pow(2,q/2)*math.gamma(q/2))*math.pow(x,q/2-1)*math.pow(math.e,-x/2)
def cdf_array(x,q):
    l = []
    for i in x:
        l.append(float(cdf(i,q)))
    return np.array(l)

In [ ]:
"""The null hypothesis is that Previous crop and Crop attribute are independent."""
sns.set(style='darkgrid',rc={"figure.figsize": (15, 8)})

x = np.linspace(0,150,150)
ax = sns.lineplot(x,cdf_array(x,q),label='chi-squared distribution function',linewidth=3.3)



ax.plot([chi2], [cdf(chi2,q)], 'o',markersize=10)
ax.text(chi2-5,cdf(chi2,q)-0.002,str(round(chi2,2)))
ax.annotate(r'$\chi^2 $ reg',xy=(chi2,0),xytext=(chi2+1,0.004),arrowprops=dict(facecolor='b', shrink=0.05))

alpha = stats.chi2.ppf(0.99,q)
ax.fill_between(x,0,cdf_array(x,q),where=x>alpha,color='r')
ax.plot([alpha], [0], 'o',markersize=10)
ax.text(alpha-2,cdf(chi2,q)-0.002,str(round(stats.chi2.ppf(0.99,q),2)))
ax.annotate(r'$ \alpha $ = 0.01',xy=(alpha,0),xytext=(alpha+5,0.005),fontsize=15,arrowprops=dict(facecolor='b', shrink=0.05))
ax.text(90,0.005,'$ H_0 $ rejection region',fontsize=20)

if p <= 0.01:
    ax.text(100,0.03,'Previous crop and Crop attribute\n are dependent at level confidence 0.01.',ha='center',wrap=True,fontsize=20)
elif p <= 0.05:
    ax.text(100,0.03,'Previous crop and Crop attribute are dependent at level confidence 0.05.',ha='center',wrap=True,fontsize=20)
else:
    ax.text(100,0.03,'Previous crop and Crop attribute are independent.',ha='center',wrap=True,fontsize=20)

""" If the null hypothesis is true, the observed and expected frequencies will be close in value and the χ2 statistic will be 
close to zero. If the null hypothesis is false, then the χ2 statistic will be large"""
ax.set_title('Chi-squared distribution (q = '+str(q)+') for Previous crop - Crop',fontsize=20,fontweight=5)
ax.grid(color='darkgrey',alpha=0.5)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_color('darkgray')
ax.spines['left'].set_color('darkgray')
figure = ax.get_figure()
#figure.savefig('chi2.png',transparent=True)
plt.show()

In [16]:
dfdates['Duration of \n season (days)'] = dfdates['Harvest date']-dfdates['Planting date']

In [17]:
dfdates['Duration of \n season (days)'] = dfdates['Duration of \n season (days)'].apply(lambda x: float(x.days))
dfdates

,Parcel,Area,Year,Crop,Seed production,Variety,Previous crop,Seed units,Manure,Fertiliser amount,...,Harvest,Seed,Fertiliser,Pesticide,Yield,Price,Revenue,Planting date,Harvest date,Duration of season (days)
0,G-1/1,63.38,2018,Maize,0,NS 205,Maize,184.000000,0.000000,237.719575,...,11853.74,9450.000000,5037.277795,15202.253080,7.811455,14.40,112484.94790,2018-06-15,2018-10-15,122.0
1,G-1/2,23.01,2018,Barley,0,Salamandre,Maize,46.000000,0.000000,238.636364,...,11762.74,8594.306050,7079.225979,1435.000000,7.146352,15.80,112912.36650,2017-09-28,2018-06-05,250.0
2,G-2/1,94.88,2018,Soybean,0,Gorstak,Maize,100.330000,0.000000,214.633115,...,11549.50,10191.216890,7146.257137,11843.414040,3.388877,37.35,126574.56120,2018-04-19,2018-09-13,147.0
3,G-2/2,20.59,2018,Soybean,0,Wendy,Maize,93.150000,0.000000,140.845070,...,11571.08,5428.848956,4776.056338,12594.378820,3.712967,37.35,138679.33460,2018-04-17,2018-09-14,150.0
4,G-3/1,63.62,2018,Maize,0,DKC 5830,Wheat,60.000000,1.886199,474.693493,...,12801.04,19899.770510,19745.513990,4947.696479,14.602326,14.40,210273.49890,2018-04-17,2018-09-14,150.0
5,G-3/2,17.80,2018,Maize,0,DKC 5830,Wheat,14.000000,2.134831,264.044944,...,12801.04,16595.812360,8427.921348,6112.939326,13.635393,14.40,196349.66290,2018-04-17,2018-09-14,150.0
6,G-4/1,63.32,2018,Soybean,0,Galeb,Maize,74.226153,0.000000,307.643714,...,11571.08,7447.109918,10072.501580,11048.218540,3.975837,37.35,148497.51260,2018-04-13,2018-09-16,156.0
7,G-4/2,24.71,2018,Soybean,0,Galeb,Maize,80.938891,0.000000,273.735330,...,11571.08,7447.109918,10072.501580,11048.218540,4.068798,37.35,151969.60740,2018-04-13,2018-09-16,156.0
8,G-5/1,76.88,2018,Maize,0,DKC 5830,Soybean,0.949532,0.000000,386.485432,...,12801.04,20071.655440,7823.860562,9002.979318,16.763788,14.40,241398.54320,2018-04-13,2018-09-16,156.0
9,G-5/2,20.98,2018,Maize,0,DKC 5830,Soybean,0.810296,0.000000,266.920877,...,12801.04,19877.951480,8530.791230,7781.716397,11.231649,14.40,161735.74830,2018-04-13,2018-09-16,156.0


In [58]:
duration_mean = dfdates['Duration of \n season (days)'].groupby(dfdates['Crop']).mean()
duration_mean

Crop
2nd soybean     117.000000
Barley          250.000000
Maize           155.533333
Oilseed rape    293.400000
Soybean         219.916667
Sunflower       138.680000
Wheat           249.575758
Name: Duration of \n season (days), dtype: float64

In [59]:
duration_mean = duration_mean.apply(lambda x: round(x,2))
duration_mean

Crop
2nd soybean     117.00
Barley          250.00
Maize           155.53
Oilseed rape    293.40
Soybean         219.92
Sunflower       138.68
Wheat           249.58
Name: Duration of \n season (days), dtype: float64

In [60]:
ddf = pd.DataFrame(columns=['Crop','Variety with max \n season','Max season'])
for i in range(n):
    df = dfdates[dfdates['Crop'] == crops[i]]
    duration_variety = df['Duration of \n season (days)'].groupby(dfdates['Variety']).mean()
    sort_duration = duration_variety.sort_values(ascending=False)
    ddf.loc[i] = [crops[i],list(sort_duration.index.values)[0],sort_duration[0]]
ddf = ddf.set_index('Crop')
ddf

,Variety with max season,Max season
Crop,,
Maize,NS 3023,166.000000
Barley,Salamandre,250.000000
Soybean,Wendy,273.714286
Wheat,Sofru + Avenue,263.000000
Sunflower,LG 56.33Cl,146.500000
Oilseed rape,Immient,293.400000
2nd soybean,Favorit,117.000000


In [61]:
dur = pd.concat([duration_mean,ddf],axis=1,sort=True)
dur.columns = ['Duration of \n season (days)','Variety with max \n season','Max season']
dur

,Duration of season (days),Variety with max season,Max season
2nd soybean,117.00,Favorit,117.000000
Barley,250.00,Salamandre,250.000000
Maize,155.53,NS 3023,166.000000
Oilseed rape,293.40,Immient,293.400000
Soybean,219.92,Wendy,273.714286
Sunflower,138.68,LG 56.33Cl,146.500000
Wheat,249.58,Sofru + Avenue,263.000000


In [63]:
styles = [dict(selector="th", props=[("font-size", "100%"),("text-align", "center"),("background-color","olivedrab"),("border-left","0.2px solid black")]),
          dict(selector="caption", props=[("text-align","center"),("font-weight","bold"),("font-size","150%")]),
         dict(selector="td", props=[("font-size", "100%"),("text-align", "center"),("background-color","#cce59a"),("border","0.2px solid black")]),
          dict(selector="th:first-child", props=[("border","0.2px solid black")]),
         dict(selector='',props=[("border-collapse","collapse"),("border","0.2px solid black")])]

stylerd = dur.style.set_table_styles(styles).set_caption('Duration of season')

stylerd

,Duration of season (days),Variety with max season,Max season
2nd soybean,117,Favorit,117
Barley,250,Salamandre,250
Maize,155.53,NS 3023,166
Oilseed rape,293.4,Immient,293.4
Soybean,219.92,Wendy,273.714
Sunflower,138.68,LG 56.33Cl,146.5
Wheat,249.58,Sofru + Avenue,263


In [ ]:
#I used my function from Homework1 
def corr(y):    
    Z = {}  #for centered attributes vectors
    for j in list(y):
        mean = y[j].sum()*(1/y[j].shape[0])
        Z[j] = [i - mean for i in y[j]] #centered vector for attribute j
        
    df_dic = {' ': list(y)}
    for j in list(y):
        Zj = np.array(Z[j])
        df_list = []
        for i in list(y):
            Zi = np.array(Z[i])
            norm_Zj = np.linalg.norm(Zj,2)
            norm_Zi = np.linalg.norm(Zi,2)
            df_list.append(np.dot(np.transpose(Zj/norm_Zj),Zi/norm_Zi)) #corr = cos(teta) for every two attributes i and j
        df_dic[j] = df_list #correlations for attribute j relative to all attributes (include corr for j,j) 
    columns = list(df_dic.keys())
    sigmaCorr = pd.DataFrame(df_dic, columns = columns)
    sigmaCorr = sigmaCorr.set_index(' ')
    sns.set(font_scale=1, rc={"figure.figsize": (8, 8)})
    ax = sns.heatmap(sigmaCorr, square=True, annot=True,annot_kws={'fontsize': 20},cmap="YlGn")
    ax.set_yticklabels(list(y),fontsize=13,rotation=0)
    ax.set_xticklabels(list(y),fontsize=13,rotation=90)
    ax.set_title('Heat map for correlation', fontsize='30',fontweight='bold')
    figure = ax.get_figure()
    #figure.savefig('correlation.png',bbox_inches='tight',transparent=True)
    return sigmaCorr

In [ ]:
corr_list = ['Yield','Seed','Fertiliser','Pesticide','Duration of \n season (days)'] #list of attributes for correlation

In [ ]:
dfcorr = dfdates[corr_list] #data frame for correlation
"""for i in corr_list:
    for j in dfcorr[i].index.values:
        if dfcorr.loc[j,i] == 0:
            dfcorr = dfcorr.drop(j,axis=0)"""

In [ ]:
dfcorr.dtypes

In [ ]:
corr(dfcorr)

In [ ]:
costs = list(data)[11:20] #costs are in database from column 11 till 19
costs

In [ ]:
dfcosts = data[costs]
for i in dfcosts['Pesticide application'].index.values:
    try:
        float(dfcosts.loc[i,'Pesticide application'].replace(',','.')) #in column Pesticide application one element is #REF! 
    except ValueError:
        dfcosts = dfcosts.drop(i,axis=0) #delete row if error occurs
        datap = data.drop(i,axis=0) #need to delete row from this df too
dfcosts['Pesticide application'] = dfcosts['Pesticide application'].apply(convert_todot) #now we can convert objects to numbers

In [ ]:
datap['Profit'] = datap['Revenue'] - dfcosts.sum(axis=1)

In [ ]:
sort = datap['Profit'].sort_values(ascending=False)

In [ ]:
parcel = []
crop_list = []
area = []
profit = []
yield_ = []
for i in sort[:5].index.values:
    profit += [round(sort.loc[i],2)]
    parcel += [datap.loc[i,'Parcel']]
    crop_list += [datap.loc[i,'Crop']]
    area += [round(datap.loc[i,'Area'],2)]
    yield_ += [round(datap.loc[i,'Yield']*1000,2)]
d = {'': [1,2,3,4,5],'Parcel': parcel,'Area (ha)': area,'Crop': crop_list,'Yield (kg/ha)': yield_,'Profit (RSD/ha)': profit}
top5 = pd.DataFrame(d,columns=['','Parcel','Area (ha)','Crop','Yield (kg/ha)','Profit (RSD/ha)'])
top5 #top 5 fields with the best profit

In [ ]:
top5 = top5.set_index('')
top5

In [ ]:
styles = [dict(selector="th", props=[("font-size", "100%"),("text-align", "center"),("background-color","olivedrab"),("border-left","0.2px solid black")]),
          dict(selector="caption", props=[("text-align","center"),("font-weight","bold"),("font-size","150%")]),
         dict(selector="td", props=[("font-size", "100%"),("text-align", "center"),("background-color","#cce59a"),("border","0.2px solid black")]),
         dict(selector='',props=[("border-collapse","collapse"),("border","0.2px solid black")])]

styler2 = top5.style.set_table_styles(styles).set_caption("Top 5 fileds with best profit")

styler2

In [ ]:
grouped_profit = datap['Profit'].groupby(datap['Crop'])
profit_mean = grouped_profit.mean()
profit_mean

In [ ]:
#I didn't used this on presentation, but this is like all in one picture
sns.set(style='darkgrid',rc={"figure.figsize": (15, 8)})
palette=['lightsteelblue','plum','lightgreen','burlywood','lightsalmon','navajowhite','gold']
ax = sns.boxplot(x='Crop',y='Profit',data=datap[['Crop','Profit']],palette=palette)

box_line_col = ['steelblue','indigo','darkgreen','saddlebrown','darkred','darkorange','gold']

for j,col in enumerate(box_line_col):
    mybox = ax.artists[j]
    mybox.set_edgecolor(col)
    for k in range(j*6,j*6+6):
        line = ax.lines[k]
        line.set_color(col)
        line.set_mfc(col)
        line.set_mec(col)

ax.set_ylabel('profit',fontsize=15)
ax.set_xlabel('crop',fontsize=15)
ax.set_xticklabels(crops,fontsize=16)
y = ax.get_yticks()
yint = range(math.floor(min(y)),math.ceil(max(y)),20000)
ax.set_yticks(yint)


ax.set_title('Profit by crop', fontsize=30,fontweight='bold')
ax.grid(color='darkgrey',alpha=0.5)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_color('darkgray')
ax.spines['left'].set_color('darkgray')

figure = ax.get_figure()
#figure.savefig('Profit by crop',transparent=True)
plt.show()

In [ ]:
sorted_profit = profit_mean.sort_values(ascending=False)
sorted_profit

In [ ]:
profit_std = grouped_profit.std()
profit_std

In [ ]:
relative = profit_std/profit_mean
relative

In [ ]:
relative.dropna(inplace=True) #for 2nd soybean std is NaN so I drop this row
relative

In [ ]:
sorted_relative = relative.sort_values(ascending=False)
sorted_relative

In [ ]:
os.chdir('D:\\Moji fajlovi\\Master\\prog\\Project 5 - Agriculture Data Analysis\\images') #changing folder for pictures

In [ ]:
#Figure with name of crop, average and relative profit and boxplot for that crop
def profit(data,i):
    palette = ['tomato','springgreen','goldenrod','lightgreen','#b3b300','yellowgreen','mediumaquamarine']
    crop = list(data['Crop'].unique())
    df = data[data['Crop']==crop[i]]
    
    grouped_profit = datap['Profit'].groupby(datap['Crop'])
    profit_mean = grouped_profit.mean()
    profit_std = grouped_profit.std()
    relative = profit_std/profit_mean
    relative.dropna(inplace=True)
    
    fig = plt.figure(figsize=(8,8),facecolor=palette[i])
    if crop[i] == '2nd soybean':
        ax = fig.add_subplot(111)
        ax.text(0.1,1.5,'A little bit of data for '+crop[i]+'! Please expand your database.',fontsize=20,fontweight='bold',color='r')
        plt.axis('off')
    else:
        gs = gridspec.GridSpec(2,2)
    
        ax = fig.add_subplot(gs[0,1])
        im = plt.imread(crop[i]+'.png')
        im[np.all(im == [0, 0, 0, 255], axis=2)] = [0, 0, 0, 0] #remove black background
        ax.imshow(im)
        ax.axis('off')
    
        ax3 = fig.add_subplot(gs[1,0])
        sns.boxplot(x='Crop',y='Profit',data=df,ax=ax3, palette=['burlywood'])
        ax3.set_facecolor(palette[i])
        ax3.spines['top'].set_visible(False)
        ax3.spines['bottom'].set_visible(False)
        ax3.spines['right'].set_visible(False)
        ax3.spines['left'].set_color('dimgray')
        ax3.tick_params(axis='y', colors='k',labelsize=15)
        ax3.tick_params(axis='x', colors='k',labelsize=15)
        ax3.set_ylabel('Profit (rsd/ha)',fontsize=15)
        ax3.set_xlabel('Crop',fontsize=15)
        ax3.grid(axis='y',color='dimgrey')
    
        plt.figtext(0.6,0.4,'Average profit \n'+str(round(profit_mean[crops[i]],2))+' RSD/ha',fontsize=25,fontweight='bold')
        plt.figtext(0.6,0.25,'Relative profit \n'+str(round(relative[crops[i]],2)),fontsize=25,fontweight='bold')
    
        fig.suptitle(crop[i],x=0.3,y=0.79,fontsize=50,fontweight='bold')
        fig.tight_layout()
        fig.savefig(crop[i]+'_profit.png',bbox_inches='tight',facecolor=palette[i])
    plt.show()

In [ ]:
for i in range(n):
    profit(datap,i)

In [ ]:
grouped_fp = data[['Fertiliser amount','Pesticide amount']].groupby(data['Crop'])
fp_mean = grouped_fp.mean()
fp_mean

In [ ]:
ranked_f = fp_mean.sort_values(by=['Fertiliser amount'],ascending=False)
ranked_f

In [ ]:
ranked_p = fp_mean.sort_values(by=['Pesticide amount'],ascending=False)
ranked_p

In [ ]:
os.chdir('D:\\Moji fajlovi\\Master\\prog\\Project 5 - Agriculture Data Analysis')

In [ ]:
fig = plt.figure(figsize=(20,8))
ax = fig.add_subplot(111)
ax2 = ax.twinx()

width = 0.4 #for bars

fp_mean['Fertiliser amount'].plot(kind='bar', color='darkgreen', ax=ax, width=width, position=1,rot=0,fontsize=20)
fp_mean['Pesticide amount'].plot(kind='bar', color='darkorange', ax=ax2, width=width, position=0,rot=0,fontsize=20)

ax.set_ylabel('fertiliser amount (kg/ha)',fontsize=15)
ax2.set_ylabel('pesticide amount (kg/ha)',fontsize=15)
ax.set_xlabel('crop',fontsize=20)
ax.legend(['Fertiliser amount'],loc='upper left', fontsize=15)
ax2.legend(['Pesticide amount'],loc='upper right',fontsize=15)
ax.spines['top'].set_visible(False)

ax2.spines['bottom'].set_color('dimgray')
ax2.spines['right'].set_color('dimgray')
ax2.spines['left'].set_color('dimgray')
ax.spines['left'].set_color('dimgray')

ax.set_facecolor('w')
ax.grid(axis='y',color='darkgrey')
ax.grid(None,axis='x')
ax2.grid(None)

#fig.savefig('fertilizer_pesticide.png',bbox_inches='tight',transparent=True)
fig.suptitle('Fertiliser and pesticide consumption (kg/ha)',x=0.5,y=0.98,fontsize=30,fontweight='medium')
plt.show()

In [ ]:
grouped_fp_byvar = data[['Fertiliser amount','Pesticide amount']].groupby(data['Variety'])
fp_byvar_mean = grouped_fp_byvar.mean()
fp_byvar_mean

In [ ]:
top3_var_p = fp_byvar_mean.sort_values(by=['Pesticide amount'],ascending=False)[:3]
top3_var_p

In [ ]:
top3_var_f = fp_byvar_mean.sort_values(by=['Fertiliser amount'],ascending=False)[:3]
top3_var_f

In [ ]:
#print appropriate crop next to variety
for i in range(n):
    data2 = data[data['Crop'] == crops[i]]
    varieties = list(data2['Variety'].unique())
    variety = list(top3_var_p.index.values)
    if variety[0] in varieties:
        top3_var_p.loc[variety[0],'Crop'] = crops[i]
    if variety[1] in varieties:
        top3_var_p.loc[variety[1],'Crop'] = crops[i]
    if variety[2] in varieties:
        top3_var_p.loc[variety[2],'Crop'] = crops[i]
columns = ['Crop','Pesticide amount','Fertiliser amount']
top3_var_p = top3_var_p[columns]
top3_var_p #for pesticide

In [ ]:
top3_var_p[['Pesticide amount','Fertiliser amount']] = top3_var_p[['Pesticide amount','Fertiliser amount']].apply(lambda x: round(x,2))
top3_var_p

In [ ]:
styles = [dict(selector="th", props=[("font-size", "100%"),("text-align", "center"),("background-color","olivedrab"),("border-left","0.2px solid black")]),
          dict(selector="caption", props=[("text-align","center"),("font-weight","bold"),("font-size","150%")]),
         dict(selector="td", props=[("font-size", "100%"),("text-align", "center"),("background-color","#cce59a"),("border","0.2px solid black")]),
          dict(selector="th:first-child", props=[("border","0.2px solid black")]),
         dict(selector='',props=[("border-collapse","collapse"),("border","0.2px solid black")])]

styler3_p = top3_var_p.style.set_table_styles(styles).set_caption('Varieties with most used pesticide')

styler3_p

In [ ]:
for i in range(n):
    data2 = data[data['Crop'] == crops[i]]
    varieties = list(data2['Variety'].unique())
    variety = list(top3_var_f.index.values)
    if variety[0] in varieties:
        top3_var_f.loc[variety[0],'Crop'] = crops[i]
    if variety[1] in varieties:
        top3_var_f.loc[variety[1],'Crop'] = crops[i]
    if variety[2] in varieties:
        top3_var_f.loc[variety[2],'Crop'] = crops[i]
columns = ['Crop','Fertiliser amount','Pesticide amount']
top3_var_f = top3_var_f[columns]
top3_var_f #for fetilizer

In [ ]:
top3_var_f[['Fertiliser amount','Pesticide amount']] = top3_var_f[['Fertiliser amount','Pesticide amount']].apply(lambda x: round(x,2))
top3_var_f

In [ ]:
styles = [dict(selector="th", props=[("font-size", "100%"),("text-align", "center"),("background-color","olivedrab"),("border-left","0.2px solid black")]),
          dict(selector="caption", props=[("text-align","center"),("font-weight","bold"),("font-size","150%")]),
         dict(selector="td", props=[("font-size", "100%"),("text-align", "center"),("background-color","#cce59a"),("border","0.2px solid black")]),
          dict(selector="th:first-child", props=[("border","0.2px solid black")]),
         dict(selector='',props=[("border-collapse","collapse"),("border","0.2px solid black")])]

styler3_f = top3_var_f.style.set_table_styles(styles).set_caption('Varieties with most used pesticide')

styler3_f

In [ ]:
avg = datap['Profit'].groupby([datap['Crop'],datap['Year']]).mean()
avg

In [ ]:
def func2(pct, allvals):
    percent = float((100*pct)/sum(allvals))
    return "{:.2f}%".format(pct,percent)

In [ ]:
def evaluate_decision(data):
    table = data[['Crop','Previous crop']].pivot_table(index='Previous crop', columns='Crop',aggfunc=len,fill_value=0)
    average_profit = data['Profit'].groupby([data['Crop'],data['Year']]).mean()
    index = list(average_profit.index.values) #list of pairs crop-year
    
    good_parcels = 0
    bad_parcels = 0
    not_enough_data = 0
    
    columns = ['Parcel','Year','Existing crop','Relaized profit','Previous crop','Max profit gain','Crop for max','Average profit \n gain']
    df = pd.DataFrame(columns=columns) #i will fill this df later
    for i in list(data.index.values):
        possible_profits = [] #list of all possible profits that could be obtained if we had picked some other crop
        pcrops = []
        crop_today = data.loc[i,'Crop'] #crop which we planted in observed year
        crop_before = data.loc[i,'Previous crop'] #previous crop
        year = data.loc[i,'Year'] #observed year
        profit_today = data.loc[i,'Profit'] #achieved profit in observed year
        
        oportunities = table.loc[crop_before]
        
        possible_crops = [c for c in list(oportunities.index.values) if oportunities[c] != 0]  # crops which could be planted a year before
        for crop_tomorrow in possible_crops: #for every possible crop which could be planted do next
            if crop_tomorrow != crop_today:
                if (crop_tomorrow,year) in index: #if it is true then we have an average profit which is our possible profit
                    possible_profit = average_profit.xs((crop_tomorrow,year))
                    possible_profits += [possible_profit]
                    pcrops += [crop_tomorrow]
        
        #i need try-except block because sometimes possible_profits is empty, in that case we don't have enough information
        try:
            possible_profits = np.array(possible_profits)
            ind = possible_profits.argmax()
            
            if profit_today >= possible_profits[ind]: #decision is good if we achieved a better profit than one we could
                good_parcels += 1
                avg = possible_profits.sum()/float(len(possible_profits)) #average possible profit
                df.loc[i] = [data.loc[i,'Parcel'],year,crop_today,round(profit_today,2),crop_before,round(possible_profits[ind],2),pcrops[ind],round(avg,2)]
            else:
                bad_parcels += 1
        except ValueError:
            not_enough_data += 1
    df.index = np.arange(1,df.shape[0]+1,1)
    return bad_parcels,good_parcels,not_enough_data, df

In [ ]:
bad_parcels, good, not_enough_data, d = evaluate_decision(datap)

In [ ]:
d #data frame with parcels which have brought good decision

In [ ]:
styles = [dict(selector="th", props=[("font-size", "100%"),("text-align", "center"),("background-color","olivedrab"),("border-left","0.2px solid black")]),
          dict(selector="caption", props=[("text-align","center"),("font-weight","bold"),("font-size","150%")]),
         dict(selector="td", props=[("font-size", "100%"),("text-align", "center"),("background-color","#cce59a"),("border","0.2px solid black")]),
          dict(selector="th:first-child", props=[("border","0.2px solid black")]),
         dict(selector='',props=[("border-collapse","collapse"),("border","0.2px solid black")])]

styler5 = d[:20].style.set_table_styles(styles).set_caption('Some parcels with good decision')

styler5

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(121)
    
pie = [bad_parcels,not_enough_data,good]
wedges, texts, autotexts = ax.pie(x=pie, labels=['Decision could be better','Not enough data','Good decision'],
                                      autopct=lambda pct: func2(pct, pie),radius=0.8)
plt.setp(autotexts,fontsize=20,fontweight='bold', color='w')
        
ax.set_title('Evaluate decision making',fontdict={'fontsize': 30, 'fontweight': 'medium', 'color': 'k'})
ax.legend(loc=1,fontsize=10)

fig.savefig('evaluation.png',bbox_inches='tight',transparent=True)
ax.axis('off')
    
plt.show()

In [ ]:
#relative share fertiliser and pesticide in all costs
grouped_c = dfcosts.sum(axis=1).groupby(datap['Crop'])
outcomes = grouped_c.mean()
outcomes

In [ ]:
#converting column Pesticide application
#it was converted but in dfcosts not in datap data frame
datap['Pesticide application'] = datap['Pesticide application'].apply(convert_todot)

In [ ]:
grouped_fp_costs = datap[['Fertiliser','Fertiliser application','Pesticide','Pesticide application']].groupby(datap['Crop'])
fp_costs = grouped_fp_costs.mean()
fp_costs

In [ ]:
relative_fertilizer = fp_costs['Fertiliser']*100 / outcomes
relative_fertilizer

In [ ]:
relative_pesticide = fp_costs['Pesticide']*100 / outcomes
relative_pesticide

In [ ]:
relative_fapp = fp_costs['Fertiliser application']*100 / outcomes
relative_fapp

In [ ]:
relative_papp = fp_costs['Pesticide application']*100 / outcomes
relative_papp

In [ ]:
sharef = pd.concat([relative_fertilizer,relative_fapp],axis=1)
sharef['Total'] = relative_fertilizer + relative_fapp
sharef.columns = ['Fertilizer','Fertilizer application','Total']
sharef = sharef.applymap(lambda x: "{:.2f}%".format(x))
sharef

In [ ]:
sharep = pd.concat([relative_pesticide,relative_papp],axis=1)
sharep['Total'] = relative_pesticide + relative_papp
sharep.columns = ['Pesticide','Pesticide application','Total']
sharep = sharep.applymap(lambda x: "{:.2f}%".format(x))
sharep

In [ ]:
styles = [dict(selector="th", props=[("font-size", "100%"),("text-align", "center"),("background-color","olivedrab"),("border-left","0.2px solid black")]),
          dict(selector="caption", props=[("text-align","center"),("font-weight","bold"),("font-size","150%")]),
         dict(selector="td", props=[("font-size", "100%"),("text-align", "center"),("background-color","#cce59a"),("border","0.2px solid black")]),
          dict(selector="th:first-child", props=[("border","0.2px solid black")]),
          dict(selector="tr td:last-child", props=[("font-weight","bold")]),
         dict(selector='',props=[("border-collapse","collapse"),("border","0.2px solid black")])]

styler4 = sharef.style.set_table_styles(styles).set_caption('Share of fertilizer costs in all costs')

styler4

In [ ]:
styles = [dict(selector="th", props=[("font-size", "100%"),("text-align", "center"),("background-color","olivedrab"),("border-left","0.2px solid black")]),
          dict(selector="caption", props=[("text-align","center"),("font-weight","bold"),("font-size","150%")]),
         dict(selector="td", props=[("font-size", "100%"),("text-align", "center"),("background-color","#cce59a"),("border","0.2px solid black")]),
          dict(selector="th:first-child", props=[("border","0.2px solid black")]),
          dict(selector="tr td:last-child", props=[("font-weight","bold")]),
         dict(selector='',props=[("border-collapse","collapse"),("border","0.2px solid black")])]

styler5 = sharep.style.set_table_styles(styles).set_caption('Share of pesticide costs in all costs')

styler5